# LSTM

- ## Preliminaries

- ### Imports

In [4]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

- ### Check CPU usage

In [5]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [6]:
get_available_gpus()

[u'/gpu:0']

----------

# I. Toy examples

- ### Test

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [7]:
#load file
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 600901
total chars: 59


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [8]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [9]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 500 ms, sys: 16 ms, total: 516 ms
Wall time: 514 ms


** 2. Build the network**

In [10]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [11]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [12]:
# Generate some sample before training to know how bad it is!
generate_text(model, 100, VOCAB_SIZE, indices_char)

2 ))99[[[1111121111121111211112111121111211112111121111211112111121111211112111121111211112111121111

'2 ))99[[[11111211111211112111121111211112111121111211112111121111211112111121111211112111121111211112'

**3. Train network**

In [14]:
nb_iteration = 0
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=100


while True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    nb_iteration += 1
    generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if nb_iteration % 10 == 0:
        print("Iteration nb : %s" %nb_iteration)
        model.save_weights('weight_02/checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_iteration))



--------------------
Epoch 1/1
108s - loss: 2.8876
6 an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an an

--------------------
Epoch 1/1
108s - loss: 2.5314
5 the the the the the the the the the the the the the the the the the the the the the the the the th

--------------------
Epoch 1/1
109s - loss: 2.3183
9 the han and and and and and and and and and and and and and and and and and and and and and and an

--------------------
Epoch 1/1
109s - loss: 2.1484
t he mand the sement the sement the ment the sement the sement the ment the sement the sement the se

--------------------
Epoch 1/1
109s - loss: 2.0074
� the here in the here and and the here in the here and and the here in the here and and the here in

--------------------
Epoch 1/1
109s - loss: 1.8874
4 the will the will the will the will the will the will the will the will the will the will the will

--------------------
Epoch 1/1
108s - loss: 1.7839
s to some of the some of the 

problem of "modern ideas," and it most before with 

--------------------
Epoch 1/1
108s - loss: 0.2436
. the religious dellarion resurt
by the work of our organs! it seems that the great mysterious of
st

--------------------
Epoch 1/1
108s - loss: 0.2332
d and even prevail, the slave of greaters or concerning the last taste of values!


2


=the objecti

--------------------
Epoch 1/1
108s - loss: 0.2210
(deceptively vere various europe, even the
great philosophy of the port of the superstituation of a 

--------------------
Epoch 1/1
108s - loss: 0.2135
xamed of "phelogo's happiness," every highly developed, there all are at least for it, with the very

--------------------
Epoch 1/1
108s - loss: 0.2069
cessive prevalence
of which at the bitterly lie to the two pleases firmly and the whole of the state

--------------------
Epoch 1/1
108s - loss: 0.1972
an lives! one can never cease word--what?

220. there is an innocence in love of its kane were exper

--------------------
Epoch 1/

] moved,[11] and we never get out of this circle, because the
belief in things[12] has been from tim

--------------------
Epoch 1/1
108s - loss: 0.1057
ce, a seat in
parliament), is at the same time the traditional had its origin is that a characterist

--------------------
Epoch 1/1
108s - loss: 0.1057
5]. all the systems of morals that may be interestion of man. aultwers
mass! what far for heavens do

--------------------
Epoch 1/1
108s - loss: 0.1042
7. in every system of morals, or
expets at all that is experies in a stile, longing in which one wou

--------------------
Epoch 1/1
108s - loss: 0.1045
and its roots have
reached down even to ourselves and our present world. it is thus, perhaps the
impIteration nb : 110


--------------------
Epoch 1/1
108s - loss: 0.1039
judices, youth, origin, the accident
of men and bools, or vengeated and franking to determining spir

--------------------
Epoch 1/1
108s - loss: 0.1048
f prevails or merely inspire
defection and cultumed and such di

KeyboardInterrupt: 

**4. Generate text**

In [ ]:
generate_text(model, 100, VOCAB_SIZE, ix_to_char)